# Subway_DB

http://www.seoulmetro.co.kr/kr/equipmentList.do?menuIdx=367 에서 지하철의 각 호선당 승강기 정보를 Crawling 한다.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

---

호선이름과 승강설비가 있는 역의 이름을 저장한다.


- function_name : line_station_name
- return value : station_name
- station_name : {line_name : [station_names]}

In [2]:
def line_station_name():
    url = "http://www.seoulmetro.co.kr/kr/equipmentList.do?menuIdx=367"
    
    # 각자의 webdriver의 위치 : 여기서는 chrome webdriver 이용
    browser = webdriver.Chrome('/Users/sony/Downloads/chromedriver_win32/chromedriver')
    browser.get(url)
    time.sleep(1)
    
    element = browser.find_element_by_tag_name("select")
    
    html = browser.page_source
    dom = BeautifulSoup(html, "lxml")
    line_links = dom.select("#line > option")

    line_name = []
    for link in line_links:
        if link.text != "선택":
            line_name.append(link.text)
    
    station_name = {}
    for i in range(len(line_name)):
        browser.find_element_by_css_selector("#line").click()
        for_line_name = "#line > option:nth-child(" + str(i+2) + ")"
        browser.find_element_by_css_selector(for_line_name).click()
        time.sleep(5)
        browser.find_element_by_css_selector("#stno").click()
        time.sleep(5)

        html = browser.page_source
        dom = BeautifulSoup(html, "lxml")
        station_links = dom.select("#stno > option")

        for_station = []
        for link in station_links:
            if link.text!= "선택":
                for_station.append(link.text)

        station_name[line_name[i]] = for_station
    
    return station_name

In [3]:
station_name = line_station_name()

In [4]:
station_name

{'1호선': ['동대문', '동묘앞', '서울역', '시청', '신설동', '제기동', '종각', '종로3가', '종로5가', '청량리'],
 '2호선': ['강남',
  '강변',
  '건대입구',
  '교대',
  '구로디지털단지',
  '구의',
  '낙성대',
  '당산',
  '대림',
  '도림천',
  '동대문역사문화공원',
  '뚝섬',
  '문래',
  '방배',
  '봉천',
  '사당',
  '삼성',
  '상왕십리',
  '서울대입구',
  '서초',
  '선릉',
  '성수',
  '시청',
  '신답',
  '신당',
  '신대방',
  '신도림',
  '신림',
  '신설동',
  '신정네거리',
  '신촌',
  '아현',
  '양천구청',
  '역삼',
  '영등포구청',
  '왕십리',
  '용답',
  '용두',
  '을지로3가',
  '을지로4가',
  '을지로입구',
  '이대',
  '잠실',
  '잠실나루',
  '잠실새내',
  '종합운동장',
  '충정로',
  '한양대',
  '합정',
  '홍대입구'],
 '3호선': ['가락시장',
  '경복궁',
  '경찰병원',
  '고속터미널',
  '교대',
  '구파발',
  '금호',
  '남부터미널',
  '녹번',
  '대청',
  '대치',
  '도곡',
  '독립문',
  '동대입구',
  '매봉',
  '무악재',
  '불광',
  '수서',
  '신사',
  '안국',
  '압구정',
  '약수',
  '양재',
  '연신내',
  '오금',
  '옥수',
  '을지로3가',
  '일원',
  '잠원',
  '종로3가',
  '지축',
  '학여울',
  '홍제'],
 '4호선': ['길음',
  '남태령',
  '노원',
  '당고개',
  '동대문',
  '동대문역사문화공원',
  '동작',
  '명동',
  '미아',
  '미아사거리',
  '사당',
  '삼각지',
  '상계',
  '서울역',
  '성신여대입구',
  '수유(강북구청)',
  '

---

저장된 호선과 역에 있는 승강기 정보를 DB에 저장한다.

In [5]:
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [6]:
engine = create_engine("sqlite:///subway_elevator.db", echo = True)
metadata = MetaData()

subway_elevator = Table('subway_elevator', metadata,
                        Column('id', Integer, primary_key =True),
                        Column('line', String, nullable = False),
                        Column('station', String, nullable = False),
                        Column('facilities', String),
                        Column('operation_section', String),
                        Column('location', String),
                        Column('service_status', String)
                    )

metadata.create_all(engine)

2018-08-14 00:02:22,713 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-08-14 00:02:22,763 INFO sqlalchemy.engine.base.Engine ()
2018-08-14 00:02:22,773 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-08-14 00:02:22,777 INFO sqlalchemy.engine.base.Engine ()
2018-08-14 00:02:22,806 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("subway_elevator")
2018-08-14 00:02:22,810 INFO sqlalchemy.engine.base.Engine ()
2018-08-14 00:02:22,821 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE subway_elevator (
	id INTEGER NOT NULL, 
	line VARCHAR NOT NULL, 
	station VARCHAR NOT NULL, 
	facilities VARCHAR, 
	operation_section VARCHAR, 
	location VARCHAR, 
	service_status VARCHAR, 
	PRIMARY KEY (id)
)


2018-08-14 00:02:22,824 INFO sqlalchemy.engine.base.Engine ()
2018-08-14 00:02:22,898 INFO sqlalchemy.engine.base.Engine COMMIT


- function name : for_subway_elevator
- input : station_name

In [7]:
def for_subway_elevator(station_name):
    url = "http://www.seoulmetro.co.kr/kr/equipmentList.do?menuIdx=367"

    # 각자의 webdriver의 위치 : 여기서는 chrome webdriver 이용
    browser = webdriver.Chrome('/Users/sony/Downloads/chromedriver_win32/chromedriver')
    browser.get(url)
    time.sleep(1)

    conn = engine.connect()
    line_name = list(station_name.keys())
    for i in range(len(line_name)):
        browser.find_element_by_css_selector("#line").click()
        for_line_name = "#line > option:nth-child(" + str(i+2) + ")"
        browser.find_element_by_css_selector(for_line_name).click()
        time.sleep(5)
        browser.find_element_by_css_selector("#stno").click()
        time.sleep(5)

        for j in range(len(station_name[line_name[i]])):
            for_station_name = "#stno > option:nth-child(" + str(j+2) + ")"
            browser.find_element_by_css_selector(for_station_name).click()
            browser.find_element_by_css_selector("#searchForm > div > a").click()
            time.sleep(2)

            html = browser.page_source
            dom = BeautifulSoup(html, "lxml")
            elevator_contents = dom.select("table tbody tr")

            for content in elevator_contents:
                elevator = []
                for_elevator = content.text.replace(' ', '')
                for_elevator = for_elevator[1 : len(for_elevator)-1]
                elevator = for_elevator.split('\n')
                
                conn.execute(subway_elevator.insert(), 
                        line = line_name[i], station = station_name[line_name[i]][j], facilities = elevator[0], operation_section = elevator[1], 
                        location = elevator[2], service_status = elevator[3]
                )

In [ ]:
for_subway_elevator(station_name)

---